In [10]:
# Options
path = ''
path = '/home/aelt/Dropbox/Apps/ShareLaTeX/peer-2-peer_markets/files/' # path to save files

### Load packages

In [11]:
import numpy as np
from cvxpy import *
from print_res_to_latex import *

## Problem data (IEEE14)

In [12]:
n = 14
Pmin = np.array([-21.7,-94.2,-47.8,-7.6,-11.2,-29.5,-9.,-3.5,-6.1,-13.5,-14.9,0.,0.,0.])
Pmax = np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,20.,50.,10.])
Pren = np.array([0.,0.,0.,14.9,9.,10.,18.5,2.,7.,5.,5.,0.,0.,0.])
a = np.array([4.,1.,2.,10.,8.,4.,9.,15.,18.,7.,6.,7.,2.,8.])
b = np.array([130.,120.,135.,125.,140.,145.,150.,135.,140.,125.,120.,25.,10.,20.])
A = np.diag(a)

## Part 1: Centralized pool-based market

In [13]:
p = Variable(n)
qimp = Variable(1)
qexp = Variable(1)
obj = 1/2 * quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            sum_entries(p) + qimp - qexp == 0, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0
            ]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path+'part1_schedule.txt','Schedule for pool-based market')
print("Import: ", qimp.value, "\tExport: ", qexp.value)
print("Price: ",constraints[0].dual_value)
write_pricing(p,constraints[0].dual_value,path + 'part1_pricing.txt','Pricing for pool-based market')
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme (with added renewables): ")
print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-3395.573129584802
Schedule: 
[[-15.00016282]
 [-50.00006667]
 [-32.50020768]
 [  7.3000002 ]
 [ -2.1999996 ]
 [-18.75149581]
 [  9.50000015]
 [ -1.49999951]
 [  0.90000026]
 [ -7.85840408]
 [ -8.33441959]
 [  6.42852987]
 [ 30.00007633]
 [  6.25008851]]
Writing table...
Done.
Import:  75.766061223 	Export:  7.98766795468e-07
Price:  -69.9999998839183
Writing pricing table...
Done.
Payments and revenues under uniform pricing scheme: 
[[ 1050.01139595]
 [ 3500.0046611 ]
 [ 2275.01453355]
 [ -511.0000134 ]
 [  153.99997198]
 [ 1312.6047045 ]
 [ -665.00000953]
 [  104.99996544]
 [  -63.00001841]
 [  550.08828458]
 [  583.40937034]
 [ -449.99709047]
 [-2100.00533947]
 [ -437.50619526]]
Payments and revenues under uniform pricing scheme (with added renewables): 
[[ 1050.01139595]
 [ 3500.0046611 ]
 [ 2275.01453355]
 [-1554.00001167]
 [ -476.00002698]
 [  612.60470566]
 [-1960.00000738]
 [  -35.00003432]
 [ -553.0000176 ]
 [  200.08828516]
 [  233.40937092]
 [ -449.99709047]
 [-2100.00533947

#### No external agent:

In [14]:
p = Variable(n)
obj = 1/2 * quad_form(p, A) + b*p 
constraints = [
            sum_entries(p) == 0, 
            Pmin + Pren<= p, 
            p <= Pmax + Pren
            ]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(p.value) 
write_schedule(p,path + 'part1_schedule_noext.txt','Schedule for pool-based market without external agent')
print("Price: ",constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value)
write_pricing(p,constraints[0].dual_value,path + 'part1_pricing_noext.txt','Pricing for pool-based market without external agent')
print("Payments and revenues under uniform pricing scheme (with added renewables): ")
print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-2462.9831849879038
Schedule: 
[[ -8.84525428]
 [-25.38114527]
 [-20.1904806 ]
 [  7.30000002]
 [ -2.19999992]
 [-12.59526718]
 [  9.50000002]
 [ -1.49999988]
 [  0.90000003]
 [ -4.34019442]
 [ -4.23020931]
 [  9.94555301]
 [ 42.3096008 ]
 [  9.32739699]]
Writing table...
Done.
Price:  -94.61893454124318
Payments and revenues under uniform pricing scheme: 
[[  836.92853536]
 [ 2401.53692292]
 [ 1910.40176265]
 [ -690.71822414]
 [  208.16164874]
 [ 1191.75076098]
 [ -898.87987957]
 [  141.92839047]
 [  -85.15704433]
 [  410.66457213]
 [  400.25789825]
 [ -941.03762951]
 [-4003.28934857]
 [ -882.54836531]]
Writing pricing table...
Done.
Payments and revenues under uniform pricing scheme (with added renewables): 
[[  836.92853536]
 [ 2401.53692292]
 [ 1910.40176265]
 [-2100.5403488 ]
 [ -643.40876213]
 [  245.56141556]
 [-2649.33016859]
 [  -47.30947861]
 [ -747.48958612]
 [  -62.43010058]
 [  -72.83677446]
 [ -941.03762951]
 [-4003.28934857]
 [ -882.54836531]]


## Part 2: Community-based market

In [15]:
p = Variable(n)
q = Variable(n)
alpha = Variable(n)
beta = Variable(n)
qimp = Variable(1)
qexp = Variable(1)
obj = 1/2*quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            p + q + alpha - beta == 0, 
            sum_entries(q) == 0, 
            sum_entries(alpha) == qimp, 
            sum_entries(beta) == qexp, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0, 
            alpha >= 0, 
            beta >= 0
            ]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part2_schedule.txt','Schedule for community-based market')
print("Community trading: ")
print(q.value)
print("Agent import: ")
print(alpha.value)
print("Agent export: ")
print(beta.value)
print("Import: ", qimp.value, "\tExport: ", qexp.value)
print("Price: ",constraints[1].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[1].dual_value)
write_pricing(p,constraints[1].dual_value,path + 'part2_pricing.txt','Pricing for community-based market')
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-3395.5726670271824
Schedule: 
[[-14.99999498]
 [-50.00006498]
 [-32.50000511]
 [  7.30000001]
 [ -2.19999997]
 [-18.74984951]
 [  9.50000001]
 [ -1.49999997]
 [  0.90000002]
 [ -7.85699148]
 [ -8.33333175]
 [  6.42856496]
 [ 29.99994761]
 [  6.24998324]]
Writing table...
Done.
Community trading: 
[[  9.58844614]
 [ 44.58852401]
 [ 27.08846409]
 [-12.71155908]
 [ -3.21155731]
 [ 13.33831949]
 [-14.91155951]
 [ -3.91155759]
 [ -6.31155778]
 [  2.44545945]
 [  2.9217779 ]
 [-11.84012693]
 [-35.41152685]
 [-11.66154605]]
Agent import: 
[[ 5.4115489 ]
 [ 5.41154102]
 [ 5.41154108]
 [ 5.41155912]
 [ 5.41155733]
 [ 5.41153007]
 [ 5.41155955]
 [ 5.41155761]
 [ 5.41155781]
 [ 5.41153208]
 [ 5.4115539 ]
 [ 5.41156202]
 [ 5.41157929]
 [ 5.41156286]]
Agent export: 
[[  5.04493728e-08]
 [  5.04493737e-08]
 [  5.04493731e-08]
 [  5.04493725e-08]
 [  5.04493725e-08]
 [  5.04493726e-08]
 [  5.04493724e-08]
 [  5.04493725e-08]
 [  5.04493725e-08]
 [  5.04493725e-08]
 [  5.04493726e-08]
 [  5.04493724e

## Changing the renewable generation of agent 7

In [16]:
Pren[6] = 0

### Pool-based

In [17]:
p = Variable(n)
qimp = Variable(1)
qexp = Variable(1)
obj = 1/2*quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            sum_entries(p) + qimp - qexp == 0, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0
            ]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part1_schedule_a7changed.txt','Schedule for pool-based market without agent 7\'s production')
print("Import: ", qimp.value, "\tExport: ", qexp.value)
print("Price: ",constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value)
write_pricing(p,constraints[0].dual_value,path + 'part1_pricing_a7changed.txt','Pricing for pool-based market without agent 7\'s production')
print("Payments and revenues under uniform pricing scheme (with added renewables): ")
print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-4917.255204027389
Schedule: 
[[-15.00006611]
 [-50.00007371]
 [-32.50006221]
 [  7.30000064]
 [ -2.19999863]
 [-18.75016758]
 [ -8.89117821]
 [ -1.4999984 ]
 [  0.90000088]
 [ -7.85730111]
 [ -8.33365445]
 [  6.42856796]
 [ 30.00003188]
 [  6.25006886]]
Writing table...
Done.
Import:  94.1538328603 	Export:  2.67610366091e-06
Price:  -69.99999960745544
Payments and revenues under uniform pricing scheme: 
[[ 1050.00462188]
 [ 3500.0051401 ]
 [ 2275.00434174]
 [ -511.00004166]
 [  153.99990311]
 [ 1312.51172325]
 [  622.38247115]
 [  104.99988739]
 [  -63.00006148]
 [  550.01107465]
 [  583.35580833]
 [ -449.99975484]
 [-2100.00222009]
 [ -437.50481762]]
Writing pricing table...
Done.
Payments and revenues under uniform pricing scheme (with added renewables): 
[[ 1050.00462188]
 [ 3500.0051401 ]
 [ 2275.00434174]
 [-1554.00003581]
 [ -476.00009336]
 [  612.51172717]
 [  622.38247115]
 [  -35.00011182]
 [ -553.00005873]
 [  200.01107661]
 [  233.35581029]
 [ -449.99975484]
 [-2100.002220

### Community-based:

In [18]:
p = Variable(n)
q = Variable(n)
alpha = Variable(n)
beta = Variable(n)
qimp = Variable(1)
qexp = Variable(1)
obj = 1/2*quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            p + q + alpha - beta == 0, 
            sum_entries(q) == 0, 
            sum_entries(alpha) == qimp, 
            sum_entries(beta) == qexp, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0, 
            alpha >= 0, 
            beta >= 0
            ]
prob = Problem(Minimize(obj), constraints)
prob.solve()
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part2_schedule_a7changed.txt','Schedule for community-based market without agent 7\'s production')
print("Community trading: ")
print(q.value)
print("Agent import: ")
print(alpha.value)
print("Agent export: ")
print(beta.value)
print("Import: ", qimp.value, "\tExport: ", qexp.value)
print("Price: ",constraints[1].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[1].dual_value)
write_pricing(p,constraints[1].dual_value,path + 'part2_pricing_a7changed.txt','Pricing for community-based market without agent 7\'s production')
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-4917.253317643439
Schedule: 
[[-15.00014873]
 [-50.00029084]
 [-32.50010782]
 [  7.30000003]
 [ -2.19999992]
 [-18.75002601]
 [ -8.89013302]
 [ -1.4999999 ]
 [  0.90000005]
 [ -7.85733201]
 [ -8.33314831]
 [  6.42853739]
 [ 30.0000864 ]
 [  6.24994693]]
Writing table...
Done.
Community trading: 
[[  8.27497221]
 [ 43.27516608]
 [ 25.77495345]
 [-14.02520535]
 [ -4.52519816]
 [ 12.02484491]
 [  2.16498842]
 [ -5.22519901]
 [ -7.62520034]
 [  1.13214559]
 [  1.60795339]
 [-13.1537445 ]
 [-36.72532459]
 [-12.97515209]]
Agent import: 
[[ 6.72517666]
 [ 6.7251249 ]
 [ 6.72515451]
 [ 6.72520546]
 [ 6.72519822]
 [ 6.72518124]
 [ 6.72514474]
 [ 6.72519905]
 [ 6.72520043]
 [ 6.72518656]
 [ 6.72519506]
 [ 6.72520725]
 [ 6.72523833]
 [ 6.7252053 ]]
Agent export: 
[[  1.39889203e-07]
 [  1.39889206e-07]
 [  1.39889204e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889202e-07]
 [  1.39889201e-

# ADMM

In [ ]:
def admm_community_based(p,q,alpha,beta,Pmax,Pmin,rhoexp = 40, tau = 30):
    '''
    Solves community-based market in a distributed manner 
    using alternating direction method of multipliers (ADMM).
    
    Input: 
        Feasible initial guesses for variables
        p:     power production (consumption of negative)
        q:     energy import from community
        alpha: energy imported
        beta:  energy exported
        
        Parameters: 
        Pmax:   maximum production
        Pmin:   minimum production
        rhoexp: price for energy exported to outside
        tau:    added price (to export price) for energy import 
        
    Output: 
        The variables
    '''
    
    